In [1]:
import json
import traceback
import pandas as pd

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.chains import LLMChain, SequentialChain

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Create the ChatOpenAI object
llm = ChatOpenAI(
    model_name = 'gpt-4o',
    temperature = 0.5
)

/home/anubrata/GiT/learning_generative_ai/MCQ_Generator/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct choice",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct choice",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct choice",
    },
}

In [4]:
TEMPLATE = """
    Text: {text}
    You are an expect MCQ maker. Given the above text, it is your job to \
    create quiz of {number} multiple choice questions for {subject} students in {tone} tone.
    Make sure the questions are not repeated and check all the questions to be conforming the text as well.
    Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
    ### RESPONSE_JSON
    {response_json}
"""

In [5]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [6]:
quiz_chain = LLMChain(
    llm= llm,
    prompt = quiz_generation_prompt,
    output_key = 'quiz',
    verbose = True
)

/home/anubrata/GiT/learning_generative_ai/MCQ_Generator/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
TEMPLATE2 = """ 
    You are an expert in English Grammar and a superstar in writing. Given a Multiple Choice Quiz for {subject} students, \
    you need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity and \
    if the quiz is not as per the cognitive and analytical abilities of the students, \
    update the quiz questions which needs to be changed and change the tone such that it perfectly fits the abilities of the students.
    Quiz MCQs:
    {quiz}
    
    Check the above quiz from an expert English writer.
"""

In [8]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables =['subject', 'quiz'], 
    template=TEMPLATE2
)

In [9]:
review_chain = LLMChain(
    llm = llm,
    prompt = quiz_evaluation_prompt,
    output_key = 'review',
    verbose = True
)

In [10]:
generate_evaluate_chain = SequentialChain(
    chains = [quiz_chain, review_chain],
    input_variables = ['text', 'number', 'subject', 'tone', 'response_json'],
    output_variables = ['quiz', 'review'],
    verbose = True
)

In [16]:
with open('../data/data.txt', 'r') as file:
    TEXT = file.read()

In [17]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [19]:
# Serialize the Python dictionary into a JSON-formatted string
print(json.dumps(RESPONSE_JSON))

{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct choice"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct choice"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct choice"}}


In [24]:
NUMBER = 5
SUBJECT = 'Machine Learning'
TONE = 'Hard'

In [25]:
# https://python.langchain.com/v0.1/docs/modules/model_io/llms/token_usage_tracking/

# How to setup token usage tracking in LangChain
with get_openai_callback() as cb:
    response = generate_evaluate_chain({
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons u

In [26]:
print(f'Total Tokens: {cb.total_tokens}')
print(f'Prompt Tokens: {cb.prompt_tokens}')
print(f'Completion Tokens: {cb.completion_tokens}')
print(f'Total cost: {cb.total_cost}')

Total Tokens: 2801
Prompt Tokens: 1516
Completion Tokens: 1285
Total cost: 0.026855


In [45]:
cleaned_quiz = response.get('quiz').strip('### RESPONSE_JSON\n```json\n')
quiz_dict = json.loads(response.get('quiz').strip('### RESPONSE_JSON\n```json\n'))
quiz_dict

{'1': {'mcq': "Who coined the term 'machine learning' in 1959?",
  'options': {'a': 'Donald Hebb',
   'b': 'Walter Pitts',
   'c': 'Arthur Samuel',
   'd': 'Tom M. Mitchell'},
  'correct': 'c'},
 '2': {'mcq': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'options': {'a': 'Cybertron',
   'b': 'Perceptron',
   'c': 'Neuronet',
   'd': 'Machina'},
  'correct': 'a'},
 '3': {'mcq': 'Which book by Canadian psychologist Donald Hebb introduced a theoretical neural structure formed by interactions among nerve cells?',
  'options': {'a': 'The Organization of Behavior',
   'b': 'Learning Machines',
   'c': 'Computing Machinery and Intelligence',
   'd': 'Pattern Classification'},
  'correct': 'a'},
 '4': {'mcq': 'According to Tom M. Mitchell, what is the formal definition of a computer program learning from experience?',
  'options': {'a': 'A computer program is said to learn from experience E with respect to some class of tasks T 

In [64]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value['mcq']
    options = ' | '.join([f"{option}: {option_value}" for option, option_value in value['options'].items()])
    correct = value['correct']
    quiz_table_data.append({'MCQ': mcq, 'Choices': options, 'Correct': correct})

In [65]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning' in 1959?",
  'Choices': 'a: Donald Hebb | b: Walter Pitts | c: Arthur Samuel | d: Tom M. Mitchell',
  'Correct': 'c'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a: Cybertron | b: Perceptron | c: Neuronet | d: Machina',
  'Correct': 'a'},
 {'MCQ': 'Which book by Canadian psychologist Donald Hebb introduced a theoretical neural structure formed by interactions among nerve cells?',
  'Choices': 'a: The Organization of Behavior | b: Learning Machines | c: Computing Machinery and Intelligence | d: Pattern Classification',
  'Correct': 'a'},
 {'MCQ': 'According to Tom M. Mitchell, what is the formal definition of a computer program learning from experience?',
  'Choices': 'a: A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P if its performance at tasks in T, as measured by P, improves wi

In [67]:
quiz = pd.DataFrame(quiz_table_data)
quiz

,MCQ,Choices,Correct
0,Who coined the term 'machine learning' in 1959?,a: Donald Hebb | b: Walter Pitts | c: Arthur S...,c
1,What was the name of the experimental 'learnin...,a: Cybertron | b: Perceptron | c: Neuronet | d...,a
2,Which book by Canadian psychologist Donald Heb...,a: The Organization of Behavior | b: Learning ...,a
3,"According to Tom M. Mitchell, what is the form...",a: A computer program is said to learn from ex...,a
4,Which of the following researchers contributed...,a: Alan Turing and Donald Hebb | b: Walter Pit...,b


In [69]:
quiz.to_csv('../Machine_Learning.csv', index=False)